In [11]:

import numpy as np
import os
import time
import cv2
import pandas as pd 
import matplotlib.pyplot as plt
from math import *
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

import argparse


In [12]:
mainPath="F:\IBM\TRAIN_SET"
destPath="F:\IBM\DEST"
    

im_size = 224
rangeLower=0
rangeUpper=100
imgsInFile=30
startFileCount=0
augmentCount=5

In [13]:
apples=995
b=1374
o=1019
pa=275
wm=475

classId=["APPLES","BANANA","ORANGE","PINEAPPLE","WATERMELON"]


In [16]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder , OneHotEncoder

inputs = layers.Input(shape=(im_size, im_size, 3))
size = (im_size, im_size)

noOfClasses = 5
imgCategory=np.array([i for i in range(noOfClasses)])
inputImg=[]
imgLabels=[]

    
for i in range(len(classId)):
    dirPath=mainPath+"\\"+str(classId[i])
    cnt=0
    for j in os.listdir(dirPath): 
        ImgPath=mainPath+"\\"+str(classId[i])+"\\"+j 
        image = cv2.imread(ImgPath)       
        if(image is not None):
            #BGR->RGB
            img2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            img3 = cv2.resize(img2, (im_size, im_size))
            img3 = img3.astype('float32') / 255.0       
            inputImg.append(img3)
            imgLabels=imgLabels+[i]
        cnt+=1
        
    

In [17]:
#data=np.array(inputImg[:60]+inputImg[apples:apples+60]+inputImg[apples+b:apples+b+60]+inputImg[apples+b+o:apples+b+o+60]+inputImg[apples+b+o+pa:apples+b+o+pa+60])
#labels=np.array(imgLabels[:60]+imgLabels[apples:apples+60]+imgLabels[apples+b:apples+b+60]+imgLabels[apples+b+o:apples+b+o+60]+imgLabels[apples+b+o+pa:apples+b+o+pa+60])
data=np.array(inputImg)
labels=np.array(imgLabels)
df = pd.DataFrame(labels,columns=["A"])

print(df)
print(type(df))
one_hot_encoded_data = pd.get_dummies(df,columns=["A"]).to_numpy()
print(one_hot_encoded_data)

      A
0     0
1     0
2     0
3     0
4     0
...  ..
4133  4
4134  4
4135  4
4136  4
4137  4

[4138 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 ...
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]]


In [18]:

print(data.shape)
print(labels.shape)
print(data.size)
print(labels.size)
print(len(inputImg))
print(len(imgLabels))
print(one_hot_encoded_data.shape)

(4138, 224, 224, 3)
(4138,)
622884864
4138
4138
4138
(4138, 5)


In [20]:
outputs = tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=True, weights=None, classes=noOfClasses)(inputs)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"] )
model.summary()
model.fit(data, one_hot_encoded_data,shuffle=True, epochs=5, batch_size=30)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 5)                2264389   
 ional)                                                          
                                                                 
Total params: 2,264,389
Trainable params: 2,230,277
Non-trainable params: 34,112
_________________________________________________________________
Epoch 1/5
138/138 [==============================] - 356s 3s/step - loss: 0.5881 - accuracy: 0.7948
Epoch 2/5
138/138 [==============================] - 352s 3s/step - loss: 0.3108 - accuracy: 0.8881
Epoch 3/5
138/138 [==============================] - 346s 3s/step - loss: 0.2590 - accuracy: 0.9113
Epoch 4/5
138/138 [==============================] - 346s 3s/step - loss: 

In [21]:
if not(os.path.exists('./WeightsDir2')):
    os.makedirs('./WeightsDir2')
model.save_weights('./WeightsDir2/weights')
    

In [35]:
testApples=266
testBanana=415
testOrange=248

In [37]:
os.listdir("F:\IBM\TEST_SET")

['APPLES', 'BANANA', 'ORANGE']

In [47]:
testPath="F:\IBM\TEST_SET"
inputImgT=[]
imgLabelsT=[]

tClassId=["APPLES","BANANA","ORANGE"]

for i in range(3):
    dirPath=testPath+"\\"+str(tClassId[i])
    #print(dirPath)
    cnt=0
    #F:\IBM\TEST_SET\APPLES
    for j in os.listdir(dirPath): 
        ImgPath=dirPath+"\\"+j 
        image = cv2.imread(ImgPath)     
        #print(ImgPath)
        if(image is not None):
            #BGR->RGB
            img2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            img3 = cv2.resize(img2, (im_size, im_size))
            img3 = img3.astype('float32') / 255.0       
            inputImgT.append(img3)
            imgLabelsT=imgLabelsT+[i]
        cnt+=1
    #print(cnt)

In [48]:
print(len(inputImgT))
print(len(imgLabelsT))

929
929


In [52]:
dataT=np.array(inputImgT)
labelsT=np.array(imgLabelsT+[3,4])
print(dataT.shape)
print(labelsT.shape)

(929, 224, 224, 3)
(931,)


In [53]:
dfT = pd.DataFrame(labelsT,columns=["A"])
print(dfT)
print(type(dfT))
one_hot_encoded_dataT = pd.get_dummies(dfT,columns=["A"]).to_numpy()
one_hot_encoded_dataT = one_hot_encoded_dataT[:-2]
print(one_hot_encoded_dataT)

     A
0    0
1    0
2    0
3    0
4    0
..  ..
926  2
927  2
928  2
929  3
930  4

[931 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 ...
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]]


In [110]:
pred=model.predict(dataT)
model.evaluate(dataT,one_hot_encoded_dataT)

23/23 [==============================] - 12s 537ms/step - loss: 2.0250 - accuracy: 0.5693


[2.025041103363037, 0.5692729949951172]